# 一. 读取用户商品特征，生成可以训练的正负样本数据

In [1]:
import pickle
import numpy as np
import pandas as pd

In [19]:
TRAING_ADMIN_PATH = 'features/train_admin_features.pkl'
TRAING_ITEM_PATH = 'features/train_item_features.pkl'
TEST_ADMIN_PATH = 'features/test_admin_features.pkl'
TEST_ITEM_PATH = 'features/test_item_features.pkl'
TRAING_DATA_PATH = '../data/train.pkl'  # 读取原始训练集
TEST_DATA_PATH = '../data/test.pkl'

In [20]:
with open(TRAING_ADMIN_PATH,'rb') as f:
    train_admin_features = pickle.load(f)
with open(TRAING_ITEM_PATH,'rb') as f:
    train_item_features = pickle.load(f)
with open(TEST_ADMIN_PATH,'rb') as f:
    test_admin_features = pickle.load(f)
with open(TEST_ITEM_PATH,'rb') as f:
    test_item_features = pickle.load(f)

In [7]:
with open(TRAING_DATA_PATH,'rb') as f:
    train_data = pickle.load(f)

In [75]:
train_item_features[:4]

,item_id,num_sell,cate_id,store_id,item_price
0,1,1,2324,10013,4501
1,2,1,3882,4485,2751
2,3,2,155,8341,656
3,5,1,1191,1949,1689


In [9]:
train_data[:3]

,buyer_country_id,buyer_admin_id,item_id,create_order_time,irank,hour,date,month,year,month-date,count,dayofweek,isweekend
0,xx,8362078,1,2018-08-10 23:49:44,12,23,10,8,2018,8-10,1,4,0
1,xx,9694304,2,2018-08-03 23:55:07,9,23,3,8,2018,8-3,1,4,0
2,yy,101887,3,2018-08-27 08:31:26,3,8,27,8,2018,8-27,1,0,0


In [53]:
train_admin_features[:3]

,buyer_admin_id,num_shop,num_item,pop_unpop_rate,item_price_sum,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3
0,1,28,27,0.000000,13387,1652,60,478.107143,232.5,18,23,16.0,0.0,12.0
1,29,42,34,0.261905,13924,4710,12,331.523810,146.5,22,32,14.0,20.0,8.0
2,71,10,8,0.300000,18547,6306,637,1854.700000,1175.0,7,7,0.0,2.0,8.0


In [68]:
def drop_lose_item(data_df,item_df, lose_item):
    length = len(list(lose_item))
    all_drop_index= []
    for i,item in enumerate(list(lose_item)):
        drop_index = data_df[data_df['item_id'] == item].index[:]
        all_drop_index.extend(list(drop_index.values))
    data_df_filter = data_df.drop(all_drop_index).reset_index(drop=True)  # 去掉item不在商品表中的数据
    data_df_filter = pd.merge(data_df_filter,item_df, how='left',on='item_id') # 将商品表中的数据添加到原始dataframe中   
    return data_df_filter

In [70]:
ITEMATTR_PKL_PATH = '../data/item_attr.pkl'
item_df = pickle.load(open(ITEMATTR_PKL_PATH, 'rb'), encoding='iso-8859-1')

In [71]:
# drop lose item
train_lose_item = set(train_data['item_id'].unique()).difference(set(item_df['item_id'].unique()))
train_filter = drop_lose_item(train_data, item_df, train_lose_item)

## 二、读取前面生成的用于模型训练的特征数据

检查是否有问题

In [92]:
TRAIN_PATH = 'model_data/train_data_pos.pkl'
with open(TRAIN_PATH,'rb') as f:
    model_train_df = pickle.load(f)

In [93]:
model_train_df[:4]

,buyer_admin_id,item_id,num_shop,num_item,pop_unpop_rate,item_price_sum,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3,num_sell,cate_id,store_id,item_price,label
0,8362078,1,12,10,0.000000,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0,1,2324,10013,4501,1
1,8362078,3346056,12,10,0.000000,16655,8742,38,1387.916667,147.5,7,10,4.0,6.0,2.0,28,1243,3185,168,1
2,2436524,3346056,21,21,0.238095,19744,9200,46,940.190476,497.0,9,17,0.0,20.0,1.0,28,1243,3185,168,1
3,2792675,3346056,25,25,0.120000,48236,19339,47,1929.440000,200.0,15,19,5.0,16.0,4.0,28,1243,3185,168,1


In [199]:
TEST_PATH = 'model_data/test_data.pkl'
with open(TEST_PATH,'rb') as f:
    model_test_df = pickle.load(f)

In [200]:
model_test_df[:4]

,buyer_admin_id,item_id,num_shop,num_item,pop_unpop_rate,item_price_sum,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3,num_sell,cate_id,store_id,item_price,label
0,1061132,189045,17,16,0.058824,6922,1033,27,407.176471,326.0,7,14,3.0,9.0,5,1,1506,11599,237,1
1,2129504,189045,7,3,0.000000,1755,269,237,250.714286,237.0,1,2,7.0,0.0,0,1,1506,11599,237,1
2,2129504,189045,7,3,0.000000,1755,269,237,250.714286,237.0,1,2,7.0,0.0,0,1,1506,11599,237,1
3,1061132,5655251,17,16,0.058824,6922,1033,27,407.176471,326.0,7,14,3.0,9.0,5,1,4016,65152,642,1


# 三、读取负样本

In [2]:
negative_sample = pd.read_csv('negative_sample.csv') # id信息

In [3]:
negative_sample = negative_sample.rename(columns={'itemid':'item_id'})

In [4]:
negative_sample[:3]

,item_id,buyer_admin_id,label
0,6997896,29.0,0
1,13042493,29.0,0
2,12019651,700.0,0


## 需要添加一列，该商品在训练集中被购买的次数

In [5]:
train_df =  pickle.load(open('../data/train.pkl', 'rb'), encoding='iso-8859-1')  # 读取训练集数据
item_df =  pickle.load(open('../data/item_attr.pkl', 'rb'), encoding='iso-8859-1')

In [6]:
item_shop_count = train_df.groupby(['item_id'])['count'].agg('sum').reset_index()  # 通过count计数

In [8]:
negative_sample_new = pd.merge(negative_sample,item_shop_count,on='item_id',how='left')  # 将购买次数添加到负样本中

In [27]:
negative_sample_new[:3]

,item_id,buyer_admin_id,label,count
0,6997896,29.0,0,1.0
1,13042493,29.0,0,2.0
2,12019651,700.0,0,1.0


In [40]:
def generate_item_features(data_df,item_df):
    buyed_item_df = pd.DataFrame(columns=['item_id','num_sell'])
    buyed_item_df['item_id'] = data_df['item_id'].unique()
#     buyed_item_df['num_sell'] = data_df.groupby(['item_id'])['count'].sum().to_list()
    buyed_item_df = pd.merge(buyed_item_df, item_df, on='item_id',how='left')
    return buyed_item_df

In [79]:
negative_item_features = pd.DataFrame(columns=['item_id'])
negative_item_features['item_id'] = negative_sample_new['item_id'].unique()
negative_item_features = pd.merge(negative_item_features,item_df, on='item_id',how='left' )
negative_item_features = pd.merge(negative_item_features,negative_sample_new[['item_id','count']], on='item_id',how='left' )
negative_item_features.drop_duplicates(inplace=True)
negative_item_features = negative_item_features.rename(columns={'count':'num_sell'})

In [80]:
negative_item_features[:3]

,item_id,cate_id,store_id,item_price,num_sell
0,6997896,2043,1,119,1.0
4,13042493,1969,1,25,2.0
7,12019651,1912,1,24,1.0


In [72]:
print('负样本商品特征大小: %s x %s'%(negative_item_features.shape[0], negative_item_features.shape[1]))

负样本商品特征大小: 2519038 x 5


In [81]:
negative_sample_new[:3]

,item_id,buyer_admin_id,label,count
0,6997896,29.0,0,1.0
1,13042493,29.0,0,2.0
2,12019651,700.0,0,1.0


In [82]:
negative_sample_merge = pd.merge(negative_sample_new, negative_item_features,on='item_id',how='outer') # 将admin和item对应上

In [83]:
negative_sample_merge[:3]

,item_id,buyer_admin_id,label,count,cate_id,store_id,item_price,num_sell
0,6997896,29.0,0,1.0,2043,1,119,1.0
1,6997896,4623210.0,0,1.0,2043,1,119,1.0
2,6997896,8320958.0,0,1.0,2043,1,119,1.0


# 四、合并正负样本

In [85]:
del negative_sample_merge['count']

In [86]:
train_admin_features[:3]

,buyer_admin_id,num_shop,num_item,pop_unpop_rate,item_price_sum,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3
0,1,28,27,0.000000,13387,1652,60,478.107143,232.5,18,23,16.0,0.0,12.0
1,29,42,34,0.261905,13924,4710,12,331.523810,146.5,22,32,14.0,20.0,8.0
2,71,10,8,0.300000,18547,6306,637,1854.700000,1175.0,7,7,0.0,2.0,8.0


In [87]:
model_train_negative_sample =pd.merge(train_admin_features,negative_sample_merge,how='outer',on=['buyer_admin_id'])

In [88]:
model_train_negative_sample[:3]

,buyer_admin_id,num_shop,num_item,pop_unpop_rate,item_price_sum,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3,item_id,label,cate_id,store_id,item_price,num_sell
0,1,28,27,0.0,13387,1652,60,478.107143,232.5,18,23,16.0,0.0,12.0,5565451,0,716,2711,501,1.0
1,1,28,27,0.0,13387,1652,60,478.107143,232.5,18,23,16.0,0.0,12.0,311727,0,2160,9435,1080,1.0
2,1,28,27,0.0,13387,1652,60,478.107143,232.5,18,23,16.0,0.0,12.0,2694440,0,600,11215,167,2.0


In [90]:
# 保存负样本文件
with open('model_data/train_data_neg.pkl', 'wb') as output_file:
        pickle.dump(model_train_negative_sample, output_file)

## 将负样本与前面生成的正样本进行合并

In [98]:
all_model_train =pd.concat([model_train_df,model_train_negative_sample])

/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [99]:
all_model_train[:3]

,buyer_admin_id,cate_id,item_id,item_price,item_price_max,item_price_mean,item_price_median,item_price_min,item_price_sum,label,num_item,num_item_cate,num_sell,num_shop,num_store_cate,pop_unpop_rate,shoptime_1,shoptime_2,shoptime_3,store_id
0,8362078,2324,1,4501,8742,1387.916667,147.5,38,16655,1,10,7,1.0,12,10,0.000000,4.0,6.0,2.0,10013
1,8362078,1243,3346056,168,8742,1387.916667,147.5,38,16655,1,10,7,28.0,12,10,0.000000,4.0,6.0,2.0,3185
2,2436524,1243,3346056,168,9200,940.190476,497.0,46,19744,1,21,9,28.0,21,17,0.238095,0.0,20.0,1.0,3185


In [126]:
all_model_train = all_model_train.fillna(0)

In [129]:
all_model_train['num_sell_1'] = all_model_train['num_sell'].apply(lambda x:int(x))
del all_model_train['num_sell']
all_model_train = all_model_train.rename(columns={'num_sell_1':'num_sell'})

In [136]:
with open('model_data/all_train_data.pkl', 'wb') as output_file: # 保存pickle文件
        pickle.dump(all_model_train, output_file)

In [144]:
with open('model_data/all_train_data.pkl','rb') as f:  # 读取pickle文件
        model_train_df_new = pickle.load(f)

In [145]:
model_train_df_new[:3]

,buyer_admin_id,cate_id,item_id,item_price,item_price_max,item_price_mean,item_price_median,item_price_min,item_price_sum,label,num_item,num_item_cate,num_shop,num_store_cate,pop_unpop_rate,shoptime_1,shoptime_2,shoptime_3,store_id,num_sell
0,8362078,2324,1,4501,8742,1387.916667,147.5,38,16655,1,10,7,12,10,0.000000,4.0,6.0,2.0,10013,1
1,8362078,1243,3346056,168,8742,1387.916667,147.5,38,16655,1,10,7,12,10,0.000000,4.0,6.0,2.0,3185,28
2,2436524,1243,3346056,168,9200,940.190476,497.0,46,19744,1,21,9,21,17,0.238095,0.0,20.0,1.0,3185,28


# 五、对得到的候选数据集Admin_id和Item_id进行清洗并提取特征信息以放入模型中进行推理

## 1. 读取🐟佬提交数据生成特征，之后使用网络进行重新排序

In [156]:
yulao_sub = pd.read_csv('../submission.csv',header=None)

In [158]:
yulao_sub.shape

(11398, 31)

In [340]:
# 将🐟佬的提交信息提取出来并构成dataframe格式，将用户信息扩充到30个维度
item_id_name = yulao_sub.iloc[:,1:].values.flatten()
admin_id_name = np.repeat(yulao_sub[0].values,30)
yulao_sub_feature = pd.DataFrame(columns=['buyer_admin_id','item_id'])
yulao_sub_feature['buyer_admin_id'] = admin_id_name 
yulao_sub_feature['item_id'] = item_id_name

In [341]:
yulao_sub_feature[:2]

,buyer_admin_id,item_id
0,152,8410857
1,152,7937154


### Merge客户信息

处理鱼佬admin_id 和item_id数据

+ 首先查看客户信息，客户信息可能存在缺失情况

In [342]:
# 之前生成的测试集上用户的特征信息，根据用户在测试集上的购买记录生成得到，已经去除了用户购买商品不在商品表中的数据
with open('features/test_admin_features.pkl','rb') as f:
    test_admin_features = pickle.load(f)  

In [343]:
# 合并信息
yulao_sub_feature = pd.merge(yulao_sub_feature, test_admin_features, on='buyer_admin_id',how='outer')
# 缺失用户ID信息
print('缺失用户id为： %s'%yulao_sub_feature[yulao_sub_feature.isna().values==True]['buyer_admin_id'].unique())

缺失用户id为： [ 262400  495938  713555 1913825 2101357]


In [354]:
yulao_sub_feature.shape

(341940, 15)

In [347]:
yulao_sub_feature[:4]  # 合并用户信息之后的数据

,buyer_admin_id,item_id,num_shop,num_item,pop_unpop_rate,item_price_sum,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3
0,152,8410857,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0
1,152,7937154,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0
2,152,8472223,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0
3,152,4016066,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0


In [349]:
# 统计缺失值情况
yulao_sub_feature.isna().sum()

buyer_admin_id         0
item_id                0
num_shop             150
num_item             150
pop_unpop_rate       150
item_price_sum       150
item_price_max       150
item_price_min       150
item_price_mean      150
item_price_median    150
num_item_cate        150
num_store_cate       150
shoptime_1           150
shoptime_2           150
shoptime_3           150
dtype: int64

 读取原始test的csv文件，查看缺失用户的购买记录

In [344]:
test_df_src = pd.read_csv('../csv/Antai_AE_round1_test_20190626.csv') 

In [345]:
test_df_src[test_df_src['buyer_admin_id'] == 262400]

,buyer_country_id,buyer_admin_id,item_id,create_order_time,irank
88847,yy,262400,10045561,2018-08-21 12:36:04,5
88848,yy,262400,10045561,2018-08-21 12:36:04,3
88849,yy,262400,10045561,2018-08-21 12:36:04,6
88850,yy,262400,10045561,2018-08-21 12:36:04,8
88851,yy,262400,10045561,2018-08-21 12:36:04,13
88852,yy,262400,10045561,2018-08-21 12:36:04,9
88853,yy,262400,10045561,2018-08-21 12:36:04,4
88854,yy,262400,10045561,2018-08-21 12:36:04,2
88855,yy,262400,10045561,2018-08-21 12:36:04,11
88856,yy,262400,10045561,2018-08-21 12:36:04,12


In [346]:
item_df[item_df['item_id'] ==10045561]

,item_id,cate_id,store_id,item_price


+ 小结：

有五个顾客id为[262400,  495938,  713555, 1913825, 2101357]他们从头至尾只买了一件商品，且该商品没有出现在商品表中，所以可以直接推断这5个客户之后还是只买该商品

这5个用户不用放入模型进行训练，所以直接删除缺失的admin_id信息

In [355]:
yulao_sub_feature.dropna(axis=0, how='any', inplace=True)  # 删除缺失值所在的行

In [358]:
yulao_sub_feature.isna().sum()

buyer_admin_id       0
item_id              0
num_shop             0
num_item             0
pop_unpop_rate       0
item_price_sum       0
item_price_max       0
item_price_min       0
item_price_mean      0
item_price_median    0
num_item_cate        0
num_store_cate       0
shoptime_1           0
shoptime_2           0
shoptime_3           0
dtype: int64

### Merge商品信息

In [351]:
# item 可能同时存在训练集和测试集中，所以同时读取两个数据集中item特征信息进行统计
with open('features/test_item_features.pkl','rb') as f:
    test_item_features = pickle.load(f)
with open('features/train_item_features.pkl','rb') as f:
    train_item_features = pickle.load(f)

In [265]:
test_item_features[:3]

,item_id,num_sell,cate_id,store_id,item_price
0,189045,1,1506,11599,237
1,10713670,1,1911,39939,101
2,10546322,1,1553,65849,40


In [359]:
yulao_sub_feature_testitem = pd.merge(yulao_sub_feature,test_item_features,on='item_id',how='left') # merge训练集中商品信息
yulao_sub_feature_testitem.isna().sum() # 统计缺失值

buyer_admin_id         0
item_id                0
num_shop               0
num_item               0
pop_unpop_rate         0
item_price_sum         0
item_price_max         0
item_price_min         0
item_price_mean        0
item_price_median      0
num_item_cate          0
num_store_cate         0
shoptime_1             0
shoptime_2             0
shoptime_3             0
num_sell             412
cate_id              412
store_id             412
item_price           412
dtype: int64

In [362]:
missing_item  = yulao_sub_feature_testitem[yulao_sub_feature_testitem.isna().values==True]['item_id'].unique()

In [369]:
print('缺失商品的个数有： %s'%missing_item.shape[0])

缺失商品的个数有： 298


In [367]:
set(item_df['item_id'].to_list()).intersection(set(missing_item))  # 所以缺失的这些商品信息就是在商品属性表中缺失的商品信息

set()

如果直接将这些缺失删除是否会影响admin的个数？

In [394]:
yulao_sub_feature_testitem_drop = yulao_sub_feature_testitem.dropna(axis=0,how='any')

查看缺失用户情况

In [377]:
set(yulao_sub_feature_testitem_drop['buyer_admin_id'].unique()).difference(set(yulao_sub_feature_testitem['buyer_admin_id'].unique()))

set()

所以用户没有减少，只是减少了一些item，所以这里就将其进行删除，该商品不放入模型进行训练，在进行提交的时候这些商品就保持原来所在的位置

In [395]:
yulao_sub_feature_testitem_drop[:4]

,buyer_admin_id,item_id,num_shop,num_item,pop_unpop_rate,item_price_sum,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3,num_sell,cate_id,store_id,item_price
0,152,8410857,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0,1.0,2686.0,6213.0,800.0
1,152,7937154,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0,1.0,2686.0,40393.0,1301.0
2,152,8472223,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0,1.0,2686.0,79168.0,1234.0
3,152,4016066,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0,1.0,3834.0,54345.0,2245.0


In [396]:
yulao_sub_feature_testitem_drop['buyer_admin_id'].unique().shape

(11393,)

修改item数据类型并生成sub pickle文件

In [397]:
yulao_sub_feature_testitem_drop['item_price'] = yulao_sub_feature_testitem_drop['item_price'].astype(np.int32)
yulao_sub_feature_testitem_drop['num_sell'] = yulao_sub_feature_testitem_drop['num_sell'].astype(np.int32)
yulao_sub_feature_testitem_drop['cate_id'] = yulao_sub_feature_testitem_drop['cate_id'].astype(np.int32)
yulao_sub_feature_testitem_drop['store_id'] = yulao_sub_feature_testitem_drop['store_id'].astype(np.int32)

/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [398]:
yulao_sub_feature_testitem_drop[:3]

,buyer_admin_id,item_id,num_shop,num_item,pop_unpop_rate,item_price_sum,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3,num_sell,cate_id,store_id,item_price
0,152,8410857,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0,1,2686,6213,800
1,152,7937154,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0,1,2686,40393,1301
2,152,8472223,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0,1,2686,79168,1234


In [399]:
yulao_sub_feature_testitem_drop['num_sell'].max()

1359

In [403]:
yulao_sub_feature_testitem_drop['label'] = -1

/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [404]:
yulao_sub_feature_testitem_drop.isna().sum()

buyer_admin_id       0
item_id              0
num_shop             0
num_item             0
pop_unpop_rate       0
item_price_sum       0
item_price_max       0
item_price_min       0
item_price_mean      0
item_price_median    0
num_item_cate        0
num_store_cate       0
shoptime_1           0
shoptime_2           0
shoptime_3           0
num_sell             0
cate_id              0
store_id             0
item_price           0
label                0
dtype: int64

In [406]:
with open('model_data/yulao_sub_data1.pkl', 'wb') as output_file: # 保存pickle文件
        pickle.dump(yulao_sub_feature_testitem_drop, output_file)